In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Step1: Necessery imports
import pandas as pd
import numpy as np
import matplotlib as mpl
import scipy as scipy
import seaborn as sns
import folium
import math
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from folium import Marker, Circle, Choropleth
from folium.plugins import MarkerCluster
from pandas import DataFrame


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Step2: Loading the data 
#Change the csv file for another brand
#Added Data types for memory optimization and faster loading
data = pd.read_csv('/kaggle/input/property-prices-in-germany-june-2021/Property Prices in Germany June 2021.csv')
            
print(data.info()) 

# Real Estate 3-D Top German Cities 
* The application helps people find the best real estate deals in the World.
* The application helps people make informed decisions.
* The application visualizes data on maps.
* Examines the average and makes predictions on various factors.
* Examines factors like affordability, price to income e.t.c.
* Draws Maps and Heatmaps.
* Creates 3-D Visualizations.
* Shows the Top 10.
* Shows the correlation between factors with heatmaps.
* Make Predictions
* Is optimized to work with large datasets.

# Describing the data

In [ ]:
#Step3:Describing the data
data.describe()

# Maps

In [ ]:
#Add Geocoding
!pip install geopy==1.22.0
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['City']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

# Position by Rank Map
* Shows the number of entries by Continent,Country e.t.c.
* Shows the Rank on the tooltip
* Rank is influenced by the affordability for the working class, because it takes to account how much property can you buy on average wage 

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Rank'])+" Position by Rank"))
v_map.add_child(mc)

# Display the map
v_map

# POSITION BY RANK - Stamentoner 

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Rank'])+" Position by Rank"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamentoner').add_to(v_map)

# Display the map
v_map

# POSITION BY RANK - Stamenwatercolor

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Rank'])+" Position by Rank"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamenwatercolor').add_to(v_map)

# Display the map
v_map

# POSITION BY RANK - OPENSTREETMAP

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Rank'])+" Position by Rank"))
v_map.add_child(mc)

#Style map
folium.TileLayer('openstreetmap').add_to(v_map)

# Display the map
v_map

# Heatmap by Rank Map
* Shows the distribution of entries by Continent,Country coloring them


In [ ]:
# Create the map
v_map_heat = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 


# Add a heatmap to the base map

folium.plugins.HeatMap(data=data[['Latitude', 'Longitude']], radius=13).add_to(v_map_heat)


# Display the map
v_map_heat

# Position by Rank - Focused Color Bubble Map 
* Blue color shows the top 25%
* Yellow color shows average 50%
* Red color shows the bottom 25%

In [ ]:
# Create a Bubble base map
m_bubble_focus = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2)

def color_producer(val):
    if val <= 8:
        return 'blue'
    if val >= 8 and val<= 16:
        return 'yellow'
    else:
        return 'red'

# Add a bubble map to the base map
for i in range(0,len(data)):
    Circle(
        location=[data.iloc[i]['Latitude'], data.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(data.iloc[i]['Rank'])).add_to(m_bubble_focus)

# Display the map
m_bubble_focus

# Position by Rank - Detailed Color Bubble Map 
* Blue color shows the Top  - Where you should to be focused and the top 20%
* Yellow - Shows the Top  - Where you should be focused and the top 20-40%
* Orange - Shows the Top  - This is the average and the top 40-60%
* Red - Shows the Top  - Where you should avoid and the bottom 20-40%
* Black - Shows the Top  - Where you should avoid and the bottom 20%

In [ ]:
# Create a Bubble base map
m_bubble_detail = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2)

def color_producer(val):
    if val <= 5:
        return 'blue'
    if val >= 1 and val<= 5:
        return 'yellow'
    if val >= 6 and val<= 11:
        return 'orange'
    if val >= 16 and val<= 21:
        return 'red'
    else:
        return 'black'

# Add a bubble map to the base map
for i in range(0,len(data)):
    Circle(
        location=[data.iloc[i]['Latitude'], data.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(data.iloc[i]['Rank'])).add_to(m_bubble_detail)

# Display the map
m_bubble_detail

# Mortgage as % of average income Map
* Shows the number of entries by Continent,Country e.t.c.
* Shows the  % of average income as mortgage on tooltip - the smaller % is the better



In [ ]:
# Create the map
v_map_mortgage = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip= str(row['MortgageAsAPercentageOfIncome'])+"%"))
                      
v_map_mortgage.add_child(mc)

# Display the map
v_map_mortgage

# Top 10 cities to Live and Mortgage 

In [ ]:
data.sort_values(by='Rank', ascending=True).head(10)

# Best cities to invest in City Centre - Gross rental yield as % Map
* Where buy-to-let property earns the most
* Buy it to rent it
* Shows the Gross rental yield as % on Map tooltip. 
* The higher number is the better one

In [ ]:

# Create the map
v_map_mortgage = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip= str(row['GrossRentalYieldCityCentre'])+"%"))
                      
v_map_mortgage.add_child(mc)

# Display the map
v_map_mortgage

# Best cities to invest in City Centre - Gross rental yield as % Map
* Blue color shows the Top  Where you should to be focused and the top 10%
* Yellow - Shows the Top - Where you should be focused and the top 10% to 20%
* Orange - Shows the Top average and the top 20% to 40%
* Red - Shows  the Average and the bottom 20%
* Black - Shows the bottom aveage and the bottom 10%

In [ ]:
# Create a Bubble base map
m_bubble_detail = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2)

def color_producer(val):
    if val >= 9:
        return 'blue'
    if val <= 8 and val>= 7:
        return 'yellow'
    if val <= 6 and val>= 5:
        return 'orange'
    if val <= 4 and val>= 3:
        return 'red'
    else:
        return 'black'

# Add a bubble map to the base map
for i in range(0,len(data)):
    Circle(
        location=[data.iloc[i]['Latitude'], data.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(data.iloc[i]['GrossRentalYieldCityCentre'])).add_to(m_bubble_detail)

# Display the map
m_bubble_detail

# Top 10 cities to invest in City Centre - Gross rental yield as % :

In [ ]:
data.sort_values(by='GrossRentalYieldCityCentre', ascending=False).head(10)

# Best cities to invest in Outside of City Centre - Gross rental yield as % Map
* Where buy-to-let property earns the most
* Shows the Gross rental yield as % on Map tooltip. 
* The higher number is the better one

In [ ]:
# Create the map
v_map_mortgage = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip= str(row['GrossRentalYieldOutsideOfCentre'])+"%"))
                      
v_map_mortgage.add_child(mc)

# Display the map
v_map_mortgage

In [ ]:
# Create a Bubble base map
m_bubble_detail = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2)

def color_producer(val):
    if val >= 9:
        return 'blue'
    if val <= 8 and val>= 7:
        return 'yellow'
    if val <= 6 and val>= 5:
        return 'orange'
    if val <= 4 and val>= 3:
        return 'red'
    else:
        return 'black'

# Add a bubble map to the base map
for i in range(0,len(data)):
    Circle(
        location=[data.iloc[i]['Latitude'], data.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(data.iloc[i]['GrossRentalYieldOutsideOfCentre'])).add_to(m_bubble_detail)

# Display the map
m_bubble_detail

# Top 10 cities to invest in Outside of City Centre - Gross rental yield as % :

In [ ]:
data.sort_values(by='GrossRentalYieldOutsideOfCentre', ascending=False).head(10)

# 3-D Graph

In [ ]:
#controls the numer of rows to be read in the dataframe
start, end = 0, 529

***'3-D Graph - X-GrossRentalYieldCityCentre,Y-GrossRentalYieldOutsideofCentre,Z-PriceToRentRatioCityCentre,Size-MortgageAsAPercentageOfIncome,Color-MortgageAsAPercentageOfIncome*****

In [ ]:


fig = go.Figure(data=go.Scatter3d(
    x=data['GrossRentalYieldCityCentre'][start:end],
    y=data['GrossRentalYieldOutsideOfCentre'][start:end],
    z=data['PriceToRentRatioCityCentre'][start:end],
    text=data['City'][start:end],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=2,
        size=data['MortgageAsAPercentageOfIncome'][start:end],
        color = data['MortgageAsAPercentageOfIncome'][start:end],
        colorscale = 'Viridis',
        colorbar_title = 'MortgageAsAPercentageOfIncome<br>',
        line_color='rgb(140, 140, 170)'
    )
))








fig.update_layout(height=800, width=800,
                  title='3-D Graph - X-GrossRentalYieldCityCentre,Y-GrossRentalYieldOutsideofCentre,Z-PriceToRentRatioCityCentre,Size-MortgageAsAPercentageOfIncome,Color-MortgageAsAPercentageOfIncome')






fig.show()

# Finding Influences - Correlation Matrix (impact relationship with numbers)

In [ ]:
#Finding correlations

corr = data.corr()

mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr = corr.mask(mask)
fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),       
    xgap=3, ygap=3,
    zmin=-1, zmax=1,
colorscale='icefire',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)
fig.update_layout(title_text='Correlation Matrix (impact relationship with numbers)',
                  title_x=0.5,
                  titlefont={'size': 20},
                  width=800, height=800,
                  xaxis_showgrid=False,
                  xaxis={'side': 'bottom'},
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed',                   
                  paper_bgcolor=None,
                  template="simple_white"
                  )
fig.show()

# Pairplots and Linear Predictions

In [ ]:
sns.pairplot(data,palette='bright')
%time

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(data, kind="reg", plot_kws={'line_kws':{'color':'red'}})
plt.show()
%time

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="GrossRentalYieldCityCentre", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="GrossRentalYieldOutsideOfCentre", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="PriceToRentRatioCityCentre", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="PriceToRentRatioOutsideOfCityCentre", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="AffordabilityIndex", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="Longitude", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="GrossRentalYieldOutsideOfCentre", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

In [ ]:
sns.jointplot(x="MortgageAsAPercentageOfIncome", y="Latitude", data=data, kind = 'reg',fit_reg= True, size = 12)
plt.show()

# Pandas Profiling

In [ ]:
import pandas_profiling
report = pandas_profiling.ProfileReport(data)
from IPython.display import display
display(report)

# Quickda Analysis

In [ ]:
pip install quickda

In [ ]:
# Importing libraries
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

In [ ]:
eda_num(data)

In [ ]:
eda_num(data, method="correlation")